In [17]:
import tensorflow as tf

In [18]:
! pip install  tflite-model-maker

In [19]:
from tflite_model_maker import configs
from tflite_model_maker import model_spec

In [20]:
 !wget --no-check-certificate \
    https://namespacelabs.com/storage/IMDBDataset.csv \
    -O /content/IMDBDataset.csv

--2021-02-25 10:39:56--  https://namespacelabs.com/storage/IMDBDataset.csv
Resolving namespacelabs.com (namespacelabs.com)... 212.1.212.137
Connecting to namespacelabs.com (namespacelabs.com)|212.1.212.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66212309 (63M) [text/csv]
Saving to: ‘/content/IMDBDataset.csv’

/content/IMDBDatase 100%[===================>]  63.14M  2.44MB/s    in 25s     

2021-02-25 10:40:22 (2.52 MB/s) - ‘/content/IMDBDataset.csv’ saved [66212309/66212309]



In [21]:
import pandas as pd
df = pd.read_csv('IMDBDataset.csv')


In [22]:
df.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [23]:
df["sentiment"].unique()

array(['positive', 'negative'], dtype=object)

In [24]:
train = df[0:30000]

In [25]:
val = df[30000:40000]

In [26]:
test = val = df[40000:50000]

In [27]:
train.to_csv("train.csv",index="False")
test.to_csv("test.csv",index="False")
val.to_csv("val.csv",index="False")

In [28]:

from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

In [33]:
mb_spec = model_spec.get('mobilebert_classifier')


In [34]:
train_data = TextClassifierDataLoader.from_csv(
      filename='train.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=True)
test_data = TextClassifierDataLoader.from_csv(
      filename='test.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=False)
val_data = TextClassifierDataLoader.from_csv(
      filename='val.csv',
      text_column='review',
      label_column='sentiment',
      model_spec=mb_spec,
      is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec,validation_data=val_data, epochs=3)


INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/3
 49/625 [=>............................] - ETA: 2:55:26 - loss: 2.1154 - test_accuracy: 0.5186

In [ ]:
loss, acc = model.evaluate(test_data)


In [ ]:
config = configs.QuantizationConfig.create_full_integer_quantization(inference_input_type=tf.uint8, inference_output_type=tf.uint8, representative_data=test_data, is_integer_only=True)


In [ ]:
model.export(export_dir=".", quantization_config=config)

In [ ]:
accuracy = model.evaluate_tflite("model.tflite", test_data)
print('TFLite model accuracy: ', accuracy)